In [31]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd

In [32]:
#path="C:/Users/FUJITSU/Desktop/arabic named entity/ANERCorp.xlsx"


In [33]:
#xlsx = pd.ExcelFile(path)

In [34]:
df=pd.read_csv("UrduNewsHeadlines.csv", header=None,  sep='\t', encoding = 'utf-8')

In [35]:
df

,0,1
0,قومی,وزیراعظم عمران خان نے کل ایسی جگہ جانے کا فیص...
1,قومی,پاکستان کا یمن کے معاملے پر موثر کردار ادا کر...
2,قومی,سابق وزیراعظم یوسف رضاگیلانی کے بھانجے کی گاڑ...
3,قومی,علیم خان کی جائیدادیں ملک سے باہر ، تفصیلات آ...
4,قومی,سرکاری افسر اکرام نوید نے50کروڑ چرائے اور شہب...
...,...,...
2170,افواج,پاک فوج کا زلزلہ متاثرین کے لئے بڑا تحفہ
2171,افواج,بھارتی آرمی چیف نے بھانڈا پھوڑ دیا
2172,افواج,پاک چائنا مشترکہ فضائی مشقیں ختم
2173,افواج,کراچی میں امن بحال، رونقیں لوٹ آئیں


In [36]:
#df = df.drop(2, 1)

In [37]:
df = df.rename(columns={0: 'label', 1: 'text'})

In [38]:
df

,label,text
0,قومی,وزیراعظم عمران خان نے کل ایسی جگہ جانے کا فیص...
1,قومی,پاکستان کا یمن کے معاملے پر موثر کردار ادا کر...
2,قومی,سابق وزیراعظم یوسف رضاگیلانی کے بھانجے کی گاڑ...
3,قومی,علیم خان کی جائیدادیں ملک سے باہر ، تفصیلات آ...
4,قومی,سرکاری افسر اکرام نوید نے50کروڑ چرائے اور شہب...
...,...,...
2170,افواج,پاک فوج کا زلزلہ متاثرین کے لئے بڑا تحفہ
2171,افواج,بھارتی آرمی چیف نے بھانڈا پھوڑ دیا
2172,افواج,پاک چائنا مشترکہ فضائی مشقیں ختم
2173,افواج,کراچی میں امن بحال، رونقیں لوٹ آئیں


In [39]:
df['label'].unique()

array(['قومی', 'کھیل', 'تفریح', 'بین الاقوامی', 'موسم', 'تعلیم', 'حادثات',
       'افواج'], dtype=object)

In [40]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [41]:
train_arr=train['text'].astype(str)
train_lbl=train['label'].astype(str)
test_arr=test['text'].astype(str)
test_lbl=test['label'].astype(str)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer

In [43]:
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)

In [44]:
import pickle
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

In [45]:
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)


In [46]:
pickle.dump(tfidf, open("tfidf.pkl", "wb"))

In [47]:
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)


In [48]:
del df
del test_arr
del train_arr

In [49]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

In [50]:
train_tfmat

<1740x4342 sparse matrix of type '<class 'numpy.float64'>'
	with 19611 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.svm import LinearSVC

In [52]:
lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [53]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [54]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(lsvm, f)
    
#with open('crf_mode.pkl', 'wb') as f2:
#    pickle,dump(crf,f2)

In [55]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [59]:
test=['فواد چوہدری کی دعوت پر سائنٹفک کمیٹی میں مفتی منیب الرحمن اور مفتی پوپلز ئی شریک نہیں ہوئے، ضد کا ہمارے پاس کوئی علاج نہیں ، فواد چوہدری چاہتے ہیں قوم ایک ہی عید منائے،دو اپریل کو لاہور میں دوسری میٹنگ میں دوبارہ دونوں کو دعوت دی جائے گی، وفاقی وزیر سائنس اینڈ ٹیکنالوجی کا خطاب']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'قومی'

In [57]:
from sklearn.metrics import  accuracy_score
from sklearn import metrics

In [58]:
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

accuracy: 0.8574712643678161


In [60]:
test=['کرکٹ ہم پاکستانیوں کے ہر درد کا علاج اور ہر غم کا مداوا ہے، بلکہ ہر مسئلے کا حل ہے۔ ملک میں پورا نظام ہلا کر رکھ دینے والا سیاسی بحران ہو، معیشت کا بیڑا غرق ہورہا ہو، منہگائی نے جان عذاب کر رکھی ہو، بدامنی نے خون خشک کر رکھا ہو']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]


'کھیل'

In [62]:
test=['ڈاکٹر لِی کورونا وائرس کا شکار ہوئے تھے اور اب ایک مقامی ہسپتال نے، جہاں ان کا علاج جاری تھا، ان کی موت کی تصدیق کر دی ہ']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'قومی'

In [63]:
test=['معروف مبلغ مولانا طارق جمیل کا کہنا ہے کہ حاکم کا ایک غلط فیصلہ ملک کو تباہ اور صحیح قدم ملک کو بچا سکتا ہے، اس وقت عمران خان حاکم ہیں اور انہیں دعا کی ضرورت ہے']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'قومی'

In [64]:
test=['امریکہ میں ایک لاکھ سے زیادہ ہلاکتوں کا خدشہ: صحت کے اعلی سرکاری ماہر']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'بین الاقوامی'